<a href="https://colab.research.google.com/github/mtsizh/bottleneck-distance-for-sigma8/blob/main/wasserstein_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title #Download datasets
#@markdown Run this block of code to download our dataset to Google Colab.
#@markdown If successful this code yields 
#@markdown * folder `alldata2` containing all data for further scripts
#@markdown * file `alldata_large.zip` -- the same data zipped; not needed for scripts, but you may want to download it to your local machine

from IPython.display import clear_output

files = ['alldata.zip', 'alldata.z01', 'alldata.z02', 'alldata.z03']
print('deleting old files if exist')
!rm -rf alldata2
!rm -f alldata_large.zip
for f in files:
  !rm -f {f}
for i,f in enumerate(files):
  print('downloading part ', i+1, ' of ', len(files))
  #result = !curl -s -S https://raw.githubusercontent.com/mtsizh/bottleneck-distance-for-sigma8/main/{f} > /dev/null && echo "TRUE"
  result = !wget -q https://raw.githubusercontent.com/mtsizh/bottleneck-distance-for-sigma8/main/{f} && echo "1" || echo "0"
  if result == ['1']:
    print('OK')
  else:
    raise Exception("ERROR WHILE DOWNLOADING DATA")
clear_output()
print("DOWNLOAD SUCCESSFUL")
print("PARTS TO ZIP")
result = !zip -qq -F alldata.zip --out alldata_large.zip 2>/dev/null && echo "1" || echo "0"
if result == ['1']:
  print('OK')
else:
  raise Exception("ERROR WHILE CREATING LARGE ZIP")
print("UNZIPPING")
result = !unzip -o -qq alldata_large.zip 2>/dev/null && echo "1" || echo "0"
if result == ['1']:
  print('OK')
else:
  raise Exception("ERROR WHILE UNZIPPING")
print('cleaning')
for f in files:
  !rm -f {f}
clear_output()
print("ALL DATA DOWNLOADED AND UNPACKED")

ALL DATA DOWNLOADED AND UNPACKED


install gudhi library with the following code (needed for persistence homology calculations)

In [31]:
!pip install ott-jax
!pip install POT
!pip install eagerpy
!pip install gudhi
!pip install KDEpy
from IPython.display import clear_output
clear_output()

The following code calculates persistence intervals. Results are saved as `persistence.json` file. Computations are lengthy, please wait until the progressbar reaches 100%. For your convenience `persistence.json.zip` is created so that you can download intermediate data to your computer.

If you want to suspend your work for a while: download the zip file and you can later reupload it to Google Colab, unzip with command `!unzip persistence.json.zip` and proceed your work.

In [20]:
#@markdown Folder with data (should be ok by default)
root_folder = "./alldata2" #@param {type: "string"}
#@markdown Choose mass filtering in % (0 to 100)
remove_masses_lower_than = 0  #@param {type: "slider", min: 0, max: 100}
remove_masses_higher_than = 100  #@param {type: "slider", min: 0, max: 100}
#@markdown Choose random filtering
choose_with_replacement = 'False' #@param ['True', 'False'] 
by_number_or_percent = 'bootstrap by number' #@param ['bootstrap by percent', 'bootstrap by number']
bootstrap_percent = 50  #@param {type: "slider", min: 0, max: 100}
bootstrap_number = 10 #@param {type: "integer"}
#@markdown Distance to measure (use or not, parameters)
use_dtm = 'False'  #@param ['True', 'False']
neighbours_dtm = 92  #@param {type: "slider", min: 2, max: 100}
order_dtm = 2  #@param {type: "slider", min: 1, max: 10}
#@markdown File to save results, use JSON only! (should be ok by default)
save_to = "persistence10.json" #@param {type: "string"}

if remove_masses_higher_than <= remove_masses_lower_than:
  raise Exception("Upper masses limit should be larger then the lower limit")

if by_number_or_percent == 'bootstrap by number' and bootstrap_number <= 0:
  raise Exception("There can be only positive number of elemetns bootstrapped")


import os
from glob import glob
import re
import numpy as np
import pandas as pd
import gudhi
from ipywidgets import IntProgress
from IPython import display
from gudhi.point_cloud.dtm import DistanceToMeasure


def read_all_data(filename, mass_filter=[0, 100], 
                  bootstrap_percent=100, bootstrap_num=None, 
                  bootstrap_replace=False):
  df = pd.read_csv(filename, sep=r'\t', header=None, engine ='python')
  df.drop(df.columns.difference([0,1,2,3]), axis=1, inplace=True)
  df.rename(columns={0:'x', 1:'y', 2:'z', 3:'m'}, inplace=True)
  min_m = df['m'].quantile(mass_filter[0]/100)
  max_m = df['m'].quantile(mass_filter[1]/100)
  df = df[df['m'].between(min_m, max_m)]
  df.drop(['m'], axis=1, inplace=True)
  all_arr = df.to_numpy()
  if not bootstrap_num:
    bootstrap_num = all_arr.shape[0]*bootstrap_percent//100
  return all_arr[np.random.choice(all_arr.shape[0], 
                                  replace=bootstrap_replace,
                                  size=bootstrap_num)]

def get_persistence_intervals(point_set, dim, use_dtm):
  if use_dtm:
    # compute the values of the DTM of parameter q
    dtm = DistanceToMeasure(k=neighbours_dtm, q=order_dtm)
    DTM_values = dtm.fit_transform(point_set)
    alpha_complex = gudhi.AlphaComplex(points=point_set, weights=DTM_values.tolist())
  else:
    alpha_complex = gudhi.AlphaComplex(points=point_set)
  simplex_tree = alpha_complex.create_simplex_tree(default_filtration_value=False)
  simplex_tree.compute_persistence()
  persistence_intervals = simplex_tree.persistence_intervals_in_dimension(dim)
  return persistence_intervals

dat_files = [y for x in os.walk(root_folder) for y in glob(os.path.join(x[0], '*.dat'))]
reg_expr = '.*\/([0-9]+)groups_([0-9]+)_new.dat'
parsed = [{'filename': f, 
           'sigma': int(re.match(reg_expr, f).group(1)), 
           'red_shift': int(re.match(reg_expr, f).group(2))
           } for f in dat_files]
bar = IntProgress(min=0, max=len(parsed))
bar.value = 0
display.display(bar)
records = []

for f in parsed:
  data = read_all_data(f['filename'], 
                       [remove_masses_lower_than, remove_masses_higher_than],
                       bootstrap_percent,
                       [None, bootstrap_number][by_number_or_percent == 'bootstrap by number'],
                       choose_with_replacement == 'True')
  for dim in [0, 1, 2]:
    I = get_persistence_intervals(data, dim, use_dtm == 'True')
    df = {'sigma8*10': f['sigma'], 
          'red_shift': f['red_shift'], 
          'dimension': dim,
          'born': I[:,0].tolist(),
          'dies': I[:,1].tolist(),
          'persists': (I[:,1] - I[:,0]).tolist()}
    records.append(df)

  bar.value += 1 
all_data = pd.DataFrame.from_records(records)

all_data.to_json(save_to)
print("JSON creation complete!")

result =!zip -qq -r {save_to}.zip {save_to} 2>/dev/null && echo "1" || echo "0"
if result == ['1']:
  print('zipped')
else:
  raise Exception("ERROR WHILE CREATING ZIP")


IntProgress(value=0, max=144)

JSON creation complete!
zipped


The following code visualizes results of the previous calculation.

In [ ]:
#@markdown File to load data (should be ok by default)
read_from = "persistence.json" #@param {type: "string"}


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_style("whitegrid")
from IPython import display

df = pd.read_json(read_from)
df = df.groupby(['sigma8*10', 'red_shift', 'dimension']).agg(lambda x: sum(list(x), [])).reset_index()


def plot_by_params(dim, sig):
  plt.figure(figsize=(12,7))
  sns.set_context("notebook", font_scale=1.5)

  plt.title("$\sigma_8 = "+str(sig/10)+" $, dimension: "+str(dim))
  plt.xlabel("Birth")
  plt.ylabel("Persistence")

  pick_df = df[(df['sigma8*10'] == sig) & (df['dimension'] == dim)]
  r_shifts = pick_df['red_shift'].unique()
  cmap = plt.cm.get_cmap('plasma')

  for idx,red_shift in enumerate(r_shifts):
    X = np.array(pick_df[pick_df['red_shift'] == red_shift]['born'].values[0])
    Y = np.array(pick_df[pick_df['red_shift'] == red_shift]['persists'].values[0])
    plt.scatter(X, Y, color=cmap(idx/len(r_shifts)), label="red shift: "+str(red_shift))

  plt.legend()
  plt.show()
  

import ipywidgets as widgets

def make_widget(field):
  return widgets.Dropdown(
    options=df[field].unique(),
    #value='2',
    description=field,
    disabled=False,
  )

from ipywidgets import interactive_output

w = [make_widget(field) for field in ['dimension', 'sigma8*10']]
ui = widgets.HBox(w)
out = widgets.interactive_output(plot_by_params, {'dim': w[0], 
                                                  'sig': w[1]})
display.display(ui, out)


The following code calculates distances between pairs of simulations. Results are stored as a json file. For your convenience archive is created as well.

Warning: takes a lot of time.

In [29]:
#@markdown File to load data (should be ok by default)
read_from = "persistence10.json" #@param {type: "string"}
#@markdown Choose metric $W_p$ (p-Wasserstein) or $W_\infty$ (Bottleneck distance)
metric_to_use = "p-Wasserstein" #@param ["Bottleneck distance", "p-Wasserstein"] {allow-input: false}
#@markdown If you choose p-Wasserstein set the order p
wasserstein_order = 1.0  #@param {type: "slider", min: 1.0, max: 5.0}
#@markdown File to save results, use JSON only! (should be ok by default)
save_to = "distances10.json" #@param {type: "string"}


import numpy as np
import pandas as pd
import gudhi
#import gudhi.hera
import gudhi.wasserstein
from ipywidgets import IntProgress
from IPython import display
import tensorflow as tf

df = pd.read_json(read_from)
bar = IntProgress(min=0, max=len(df.index))
bar.value = 0
bar1 = IntProgress(min=0, max=len(df.index))
bar1.value = 0
display.display(bar, bar1)
records = []

for r_shift in df['red_shift'].unique():
  df1 = df[df['red_shift'] == r_shift]
  for dimension in df1['dimension'].unique():
    df2 = df1[df1['dimension'] == dimension]
    distances = {}
    for idx1 in range(len(df2.index)):
      I1 = np.transpose([df2.iloc[idx1]['born'], df2.iloc[idx1]['dies']]).astype(float)
      I1 = tf.convert_to_tensor(I1, dtype=float)
      sigma81 = df2.iloc[idx1]['sigma8*10']
      bar1.max = len(df2.index)
      bar1.value = 0
      for idx2 in range(len(df2.index)):
        bar1.value += 1
        if idx2 <= idx1:
          continue
        I2 = np.transpose([df2.iloc[idx2]['born'], df2.iloc[idx2]['dies']]).astype(float)
        I2 = tf.convert_to_tensor(I2, dtype=float)
        sigma82 = df2.iloc[idx2]['sigma8*10']
        #print(I1.shape, I2.shape)
        if metric_to_use == "p-Wasserstein":
          #dist = float(gudhi.hera.wasserstein_distance(np.array(I1, dtype=float), 
          #                                             np.array(I2, dtype=float)))
          dist = float(gudhi.wasserstein.wasserstein_distance(I1, 
                                                              I2,
                                                              matching=False,
                                                              enable_autodiff=True, 
                                                              keep_essential_parts=False,
                                                              order=wasserstein_order))
        else:
          dist = float(gudhi.bottleneck_distance(I1, I2))
        dsigma = np.abs(sigma81 - sigma82)
        if not dsigma in distances:
          distances[dsigma] = []
        distances[dsigma].append(dist)
      bar.value += 1
    for dsigma in distances:
      row = {'Dsigma8*10': dsigma, 
             'red_shift': r_shift, 
             'dimension': dimension,
             'distances': distances[dsigma]}
      records.append(row)

result = pd.DataFrame.from_records(records)
result.to_json(save_to)
print("JSON creation complete!")

result =!zip -qq -r {save_to}.zip {save_to} 2>/dev/null && echo "1" || echo "0"
if result == ['1']:
  print('zipped')
else:
  raise Exception("ERROR WHILE CREATING ZIP")

IntProgress(value=0, max=432)

IntProgress(value=0, max=432)

JSON creation complete!
zipped


The following code visualizes results of the previous calculation.

In [30]:
#@markdown File to load data (should be ok by default)
read_from = "distances10.json" #@param {type: "string"}

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_style("whitegrid")
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from sklearn.neighbors import KernelDensity
from KDEpy import FFTKDE

df = pd.read_json(read_from)

legend_elements = [
                    Line2D([0], [0], marker='o', color='w', 
                          label='$0$ homologies',
                          markerfacecolor='r', markersize=10),
                   Line2D([0], [0], marker='o', color='w', 
                          label='$1$ homologies',
                          markerfacecolor='b', markersize=10),
                  Line2D([0], [0], marker='o', color='w', 
                          label='$2$ homologies',
                          markerfacecolor='g', markersize=10)
                   ]

def plot_data(ax, data, box_color='b', shift=0.0):
  ax.set_xlabel("$\Delta \sigma_8$")
  ax.set_ylabel("$W$ distance")
  y = data['distances'].tolist()
  x = data['Dsigma8*10'].tolist()  
  y = [y[i] for i in np.argsort(x)]
  xu = np.sort(x)/10
  medians = [np.median(yd) for yd in y]
  import warnings
  warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
  ax.boxplot(y, 
              showmeans=True, positions=xu + shift, widths=0.006,
              boxprops=dict(color=box_color),
              whiskerprops=dict(color=box_color),
              capprops=dict(color=box_color),
              flierprops=dict(color=box_color, markeredgecolor=box_color))
  ax.plot(xu + shift, medians, c=box_color)
  ax.set_xlim(np.min(xu)-0.025, np.max(xu)+0.025)
  ax.set_navigate(False)
  ax.set_xticks(xu)
  ax.set_xticklabels(xu)

def plot_pdf(ax, data, pdf_color, limits, label):
  kde = FFTKDE(bw='silverman', kernel='gaussian')
  kde.fit(data)(None)
  b_w = kde.bw
  kde = KernelDensity(bandwidth=b_w, kernel='gaussian')
  kde.fit(data.reshape(-1, 1))
  x_d = np.linspace(limits[0], limits[1], 1001)
  logprob = kde.score_samples(x_d.reshape(-1, 1))
  plt.plot(data, np.full_like(data, 0.0), markeredgewidth=1, color=pdf_color)
  ax.set_title('Pdf approximation')
  ax.set_xlabel('$W$ distance for Homologies', fontsize=18)
  ax.set_ylabel('Probability density function', fontsize=18)
  ax.fill_between(x_d, np.exp(logprob), alpha=0.3, label=label, color=pdf_color)

def plot_by_params(red_s, Dsigma8):
  sns.set_context("notebook", font_scale=1.5)
  fig, axs = plt.subplots(1, 2, figsize=(20,6))
  df_filtered = df[df['red_shift'] == red_s]
  dims = np.unique(df_filtered['dimension'])
  colors = {0:'r', 1:'b', 2:'g'}
  for d in dims:
    data = df_filtered[df_filtered['dimension'] == d]
    plot_data(axs[0], data, 
              box_color=colors[d],
              shift={0:-0.005, 1:0.0, 2:+0.005}[d])
  axs[0].legend(handles=legend_elements, loc='upper left', prop={'size': 12})
  Dsigma810 = int(Dsigma8*10)
  df_filtered = df_filtered[df_filtered['Dsigma8*10'] == Dsigma810]
  limits=[np.min(df_filtered['distances'].tolist()), np.max(df_filtered['distances'].tolist())]
  for d in dims:
    data = df_filtered[df_filtered['dimension'] == d]['distances']
    plot_pdf(axs[1], np.array(data.tolist()[0]), pdf_color=colors[d], limits=limits, label=str(d)+" homology")  
  axs[1].legend()
  plt.show()
  
import ipywidgets as widgets

def make_widget(options, descr):
  return widgets.Dropdown(
    options=options,
    #value='2',
    description=descr,
    disabled=False,
  )

from ipywidgets import interactive_output

w = [make_widget(np.unique(df['red_shift']), 'red shift'),
     make_widget(np.unique(df['Dsigma8*10'])/10, 'Delta sigma8 x 10$')]
ui = widgets.HBox(w)
out = widgets.interactive_output(plot_by_params, {'red_s': w[0],
                                                  'Dsigma8': w[1]})
display.display(ui, out)



Output()

# Test Section